In [1]:
#https://dev.pokemontcg.io/dashboard
#https://github.com/PokemonTCG/pokemon-tcg-sdk-python

In [1]:
API_KEY = "1321d2cc-d928-4480-8d30-4e355eebc3aa"

In [4]:
!pip install pokemontcgsdk

You should consider upgrading via the 'c:\users\maxpr\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [5]:
from pokemontcgsdk import RestClient

RestClient.configure(API_KEY)

In [6]:
from pokemontcgsdk import Card
cards = Card.where(q='set.name:generations supertype:pokemon')

In [10]:
single_card = cards[0]

In [16]:
single_card.__dict__

{'abilities': None,
 'artist': 'Eske Yoshinob',
 'ancientTrait': None,
 'attacks': [Attack(name='Frog Hop', cost=['Grass', 'Colorless', 'Colorless'], convertedEnergyCost=3, damage='40+', text='Flip a coin. If heads, this attack does 40 more damage.'),
  Attack(name='Poison Impact', cost=['Grass', 'Grass', 'Colorless', 'Colorless'], convertedEnergyCost=4, damage='80', text="Your opponent's Active Pokémon is now Asleep and Poisoned.")],
 'cardmarket': Cardmarket(url='https://prices.pokemontcg.io/cardmarket/g1-1', updatedAt='2022/02/11', prices=CardmarketPrices(averageSellPrice=6.86, lowPrice=2.99, trendPrice=5.93, germanProLow=None, suggestedPrice=None, reverseHoloSell=None, reverseHoloLow=None, reverseHoloTrend=6.21, lowPriceExPlus=4.49, avg1=6.0, avg7=6.85, avg30=7.0, reverseHoloAvg1=6.99, reverseHoloAvg7=5.52, reverseHoloAvg30=5.03)),
 'convertedRetreatCost': 4,
 'evolvesFrom': None,
 'flavorText': None,
 'hp': '180',
 'id': 'g1-1',
 'images': CardImage(small='https://images.pokemontc

In [20]:
print(single_card.name)
print(single_card.nationalPokedexNumbers)
print(single_card.subtypes)
print(single_card.rarity)

Venusaur-EX
[3]
['Basic', 'EX']
Rare Holo EX


## For japanese

In [2]:
base_url = "https://jp.pokellector.com/sets"

In [3]:
from selenium import webdriver
import time
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import pandas as pd


In [4]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options,executable_path="C:\\Users\\maxpr\\Downloads\\chromedriver_win32\\chromedriver.exe" )
# driver.implicitly_wait(10)
data = []

C:\Users\maxpr\AppData\Local\Temp\ipykernel_14428\2901737340.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=chrome_options,executable_path="C:\\Users\\maxpr\\Downloads\\chromedriver_win32\\chromedriver.exe" )


In [5]:
from bs4 import BeautifulSoup

In [6]:
driver.get(base_url)

In [10]:
soup = BeautifulSoup(driver.page_source)

In [11]:
series_bs = soup.find_all('h1', class_="icon set")
blocks_bs = soup.find_all('div', class_="buttonlisting")

In [12]:
series = driver.find_elements_by_class_name('set')
blocks = driver.find_elements_by_class_name('buttonlisting')

C:\Users\maxpr\AppData\Local\Temp\ipykernel_22504\2139438567.py:1: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  series = driver.find_elements_by_class_name('set')
C:\Users\maxpr\AppData\Local\Temp\ipykernel_22504\2139438567.py:2: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  blocks = driver.find_elements_by_class_name('buttonlisting')


In [1]:
from clickhouse_driver import Client

In [2]:
db = Client("localhost")

In [3]:
extensions = db.query_dataframe(f"SELECT * FROM pokemon.extensions")

In [12]:
class DBInfo:
    """
    Represents the info of the DB
    """

    NAME = "pokemon"


class ExtensionsMeta(type):
    """
    Represents the meta class for this table.

    Basically what it does is change the string representation of the class when the class is directly used as a string.
    (think calling str(extensions) instead of calling str(extensions())).
    """

    def __repr__(self) -> str:
        """Represent name of the class, which is the name of the table."""
        return f"{DBInfo.NAME}.extensions"


class Extensions(metaclass=ExtensionsMeta):
    """This object is a simple enumeration of the name linked to the extensions table."""
    SERIE_NAME = "serieName"
    EXTENSION_NAME = "extensionName"
    EXTENSION_CODE = "extensionCode"
    EXTENSION_URL = "extensionUrl"
    EXTENSION_IMAGE_URL = "extensionImageUrl"
    EXTENSION_RELEASE_DATE = "extensionReleaseDate"
    EXTENSION_CARD_NUMBER = "extensionCardNumber"
    
class CollectionMeta(type):
    """
    Represents the meta class for this table.
    """

    def __repr__(self) -> str:
        """Represent name of the class, which is the name of the table."""
        return f"{DBInfo.NAME}.collection"


class Collection(metaclass=CollectionMeta):
    """This object is a simple enumeration of the name linked to the collection table."""
    CARD_NAME = "cardName"
    CARD_NUMBER = "cardNumber"
    CARD_EXTENSION_CODE = "cardExtensionCode"
    OWNED = "owned"
    
class CardListMeta(type):
    """
    Represents the meta class for this table.
    """

    def __repr__(self) -> str:
        """Represent name of the class, which is the name of the table."""
        return f"{DBInfo.NAME}.cardList"


class CardList(metaclass=CardListMeta):
    """This object is a simple enumeration of the name linked to the cardList table."""
    CARD_NAME = "cardName"
    CARD_JAPANESE_NAME = "cardJapaneseName"
    CARD_IMAGE_URL = "cardImageUrl"
    CARD_EXTENSION_CODE = "cardExtensionCode"
    CARD_NUMBER = "cardNumber"
    CARD_RARITY = "cardRarity"

In [21]:
def percent_owned_per_extension(db):
    df = db.query_dataframe(f"SELECT {Collection.OWNED},{Collection.CARD_EXTENSION_CODE} FROM {CardList} as cl LEFT JOIN {Collection} as co "
                                    f"ON cl.{CardList.CARD_NAME}=co.{Collection.CARD_NAME} AND"
                                    f" cl.{CardList.CARD_NUMBER}=co.{Collection.CARD_NUMBER} AND"
                                    f" cl.{CardList.CARD_EXTENSION_CODE}=co.{Collection.CARD_EXTENSION_CODE}")

    test = df[df[Collection.OWNED] == 1].groupby(Collection.CARD_EXTENSION_CODE).count()
    group = test[test[Collection.OWNED] != 0]
    return group

def get_all_extensions_for_ui(db):
    df_ext = db.query_dataframe(
        f"SELECT  {Extensions.EXTENSION_NAME}, {Extensions.EXTENSION_CODE},{Extensions.EXTENSION_IMAGE_URL}, {Extensions.EXTENSION_CARD_NUMBER}, {Extensions.EXTENSION_RELEASE_DATE} FROM {Extensions} ORDER BY {Extensions.EXTENSION_RELEASE_DATE} DESCENDING")
    group = percent_owned_per_extension(db)
    display(group)
    

    df_ext.set_index(Extensions.EXTENSION_CODE, inplace=True)
    df_ext['ownedNumber'] = group
    df_ext['ownedNumber'].fillna(0, inplace=True)
    df_ext.reset_index(inplace=True)
    df_ext['percentage'] = df_ext['ownedNumber']*100 / df_ext['extensionCardNumber']
    return df_ext

In [22]:
get_all_extensions_for_ui(db)

,owned
cardExtensionCode,
S10D,4
S8AP,12
SI,5


,extensionCode,extensionName,extensionImageUrl,extensionCardNumber,extensionReleaseDate,ownedNumber,percentage
0,S10D,Time Gazer Set,https://den-media.pokellector.com/logos/Time-G...,88,2022-04-08 01:05:00+08:00,4.0,4.545455
1,S10P,Space Juggler Set,https://den-media.pokellector.com/logos/Space-...,88,2022-04-08 01:05:00+08:00,0.0,0.000000
2,S9A,Battle Region Set,https://den-media.pokellector.com/logos/Battle...,93,2022-02-25 01:05:00+08:00,0.0,0.000000
3,S9,Star Birth Set,https://den-media.pokellector.com/logos/Star-B...,127,2022-01-14 01:05:00+08:00,0.0,0.000000
4,SI,Start Deck 100 Set,https://den-media.pokellector.com/logos/Start-...,166,2021-12-17 01:05:00+08:00,5.0,3.012048
...,...,...,...,...,...,...,...
153,B05,Leader's Stadium Set,https://den-media.pokellector.com/logos/Leader...,96,1998-10-24 01:05:00+08:00,0.0,0.000000
154,B04,Rocket Gang Set,https://den-media.pokellector.com/logos/Rocket...,65,1997-11-21 01:05:00+08:00,0.0,0.000000
155,B03,Mystery of the Fossils Set,https://den-media.pokellector.com/logos/Myster...,48,1997-06-21 01:05:00+08:00,0.0,0.000000
156,B02,Pokemon Jungle Set,https://den-media.pokellector.com/logos/Pokemo...,48,1997-03-05 01:05:00+08:00,0.0,0.000000


In [18]:
from datetime import datetime, timedelta
from clickhouse_driver import Client
import pandas as pd


class DBHandler:

    def __init__(self, domain: str):
        self.client = Client(domain)

    def list_already_present_extensions(self):
        """Return the extensions code already present in the database.

        :rtype: List
        :return: List of String EXTENSION_CODE. 
        """
        return [val[0] for val in self.client.execute(f"SELECT DISTINCT extensionCode FROM pokemon.extensions")]

    def insert_extensions(self, dataframe: pd.DataFrame):
        """Insert into the database extensions that don't alreay exists.

        :param dataframe: Dataframe to insert containing extensions
        """
        # Declare 2 filters. Only take extensions not in DB, and that have released for more than 1 week.
        dataframe.to_csv('data.csv')
        already_present = self.list_already_present_extensions()
        latest_release_date = datetime.now() + timedelta(days=7)

        filtered_df = dataframe[(~dataframe["extensionReleaseDate"].isin(already_present)) & (dataframe["extensionReleaseDate"] < latest_release_date)]
        return self.client.execute(f"INSERT INTO pokemon.extensions VALUES", filtered_df.to_dict("records"), types_check=True)

    def get_oldest_extension(self):
        """Return the extensions code already present in the database.
        :rtype: List
        :return: List of String EXTENSION_CODE.
        """
        return [val[0] for val in self.client.execute(f"SELECT MAX(extensionReleaseDate) FROM pokemon.extensions")]


In [146]:
db.execute("ALTER TABLE pokemon.extensions DELETE where 1=1")

[]

In [26]:
db_handler = DBHandler("localhost")

NameError: name 'DBHandler' is not defined

In [3]:
db_handler.list_already_present_extensions()
df = pd.read_csv('../pokemon_data_scraper/data.csv')
df['extensionReleaseDate'] = df['extensionReleaseDate'].apply(lambda x: datetime.fromisoformat(x))

In [26]:
db_handler.get_oldest_extension()[0]

datetime.datetime(2022, 4, 8, 1, 5, tzinfo=<DstTzInfo 'Asia/Singapore' +08+8:00:00 STD>)

In [58]:
for i in range(2):
    for k in range(30):
        print(i, k)
        if k == 2:
            break

0 0
0 1
0 2
1 0
1 1
1 2


In [63]:
pd.DataFrame([],columns=["test", "nothing"])

,test,nothing


In [8]:
db.execute(f"INSERT INTO pokemon.extensions VALUES", df.to_dict("records"), types_check=True)

Error on localhost:9000 ping: Unexpected EOF while reading bytes
Connection was closed, reconnecting.


158

In [15]:
" ".join(test_1.split("\n")[-2:])

NameError: name 'test_1' is not defined

In [10]:
" ".join(test_2.split("\n")[-2:])

'Feb 25th 2022'

In [16]:
!pip install beautifulsoup4

You should consider upgrading via the 'C:\Users\maxpr\Documents\MyCode\pokemon-tcg-tracker\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [7]:
from bs4 import BeautifulSoup

In [131]:
type(bs_shit.find_all('div'))

bs4.element.ResultSet

In [8]:
from joblib import Parallel, delayed

In [9]:
import re

def test(extension):
    ext_code = extension.get_attribute('name')
    ext_url = extension.get_attribute('href')
    ext_name = extension.get_attribute('title')
    ext_image_url = extension.find_element_by_css_selector('img').get_attribute('src')
    print(ext_code)
    print(ext_url)
    print(ext_name)
    
    new_driver = webdriver.Chrome(options=chrome_options,executable_path="C:\\Users\\maxpr\\Downloads\\chromedriver_win32\\chromedriver.exe" )
    new_driver.get(ext_url)
    card_metadata = new_driver.find_element_by_class_name("setinfo").text
    #print(card_metadata)
    
    date = " ".join(card_metadata.split("\n")[-2:])
    
    new_driver.quit()
    #date_of_release = timestring.Date(date).date
    
    #card_number_str = " ".join(card_metadata.split("\n")[1:3]).split("+")
    #card_number = sum([int(re.sub(r'[^0-9]','',t)) for t in card_number_str])
    
    #print(f"Released in {date} with {card_number} cards")
    return [series[idx].text, ext_code, ext_url, ext_name, ext_image_url]

def test_bs(extension_bs):
    ext_code = extension_bs['name']
    ext_url = extension_bs['href']
    ext_name = extension_bs['title']
    ext_image_url = extension_bs.find('img')['src']
    print(ext_code)
    print(ext_url)
    print(ext_name)
    
    req = requests.get(base_url+ext_url.replace("/sets",""))
    bs_shit = BeautifulSoup(req.text)
    card_metadata = bs_shit.find('div', class_="setinfo").text
    card_number_str = " ".join(card_metadata.split("\n")[3:6]).split("+")
    card_number = sum([int(re.sub(r'[^0-9]','',t)) for t in card_number_str])
    #print(card_metadata)
    
    date = " ".join(card_metadata.split("\n")[-4:]).strip()
    print("nani")
    return [series[idx].text, ext_code, ext_url, ext_name, ext_image_url]

   

In [153]:

driver = new_driver = webdriver.Chrome(options=chrome_options,executable_path="C:\\Users\\maxpr\\Downloads\\chromedriver_win32\\chromedriver.exe" )
driver.get("https://jp.pokellector.com/sets/EBB-EX-Battle-Boost")
bs = BeautifulSoup(driver.page_source)
all_cards = bs.find_all('div' , class_="card")

C:\Users\maxpr\AppData\Local\Temp\ipykernel_22504\578358483.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = new_driver = webdriver.Chrome(options=chrome_options,executable_path="C:\\Users\\maxpr\\Downloads\\chromedriver_win32\\chromedriver.exe" )


In [165]:
[card.find('a')['href'] for card in all_cards]

['/card/Shuckle-EX-Battle-Boost-EBB-1',
 '/card/Snivey-EX-Battle-Boost-EBB-2',
 '/card/Servine-EX-Battle-Boost-EBB-3',
 '/card/Serperior-EX-Battle-Boost-EBB-4',
 '/card/Sewaddle-EX-Battle-Boost-EBB-5',
 '/card/Swadloon-EX-Battle-Boost-EBB-6',
 '/card/Leavanny-EX-Battle-Boost-EBB-7',
 '/card/Virizion-EX-Battle-Boost-EBB-8',
 '/card/Charmander-EX-Battle-Boost-EBB-9',
 '/card/Charmeleon-EX-Battle-Boost-EBB-10',
 '/card/Charizard-EX-Battle-Boost-EBB-11',
 '/card/Vulpix-EX-Battle-Boost-EBB-12',
 '/card/Ninetales-EX-Battle-Boost-EBB-13',
 '/card/Moltres-EX-Battle-Boost-EBB-14',
 '/card/Victini-EX-Battle-Boost-EBB-15',
 '/card/Victini-EX-EX-Battle-Boost-EBB-16',
 '/card/Tepig-EX-Battle-Boost-EBB-17',
 '/card/Pignite-EX-Battle-Boost-EBB-18',
 '/card/Emboar-EX-Battle-Boost-EBB-19',
 '/card/Reshiram-EX-Battle-Boost-EBB-20',
 '/card/Reshiram-EX-EX-Battle-Boost-EBB-21',
 '/card/Magikarp-EX-Battle-Boost-EBB-22',
 '/card/Gyarados-EX-Battle-Boost-EBB-23',
 '/card/Articuno-EX-Battle-Boost-EBB-24',
 '/

In [192]:
card_r = requests.get(base_url.replace("/sets","")+'/card/Shuckle-EX-Battle-Boost-EBB-1')

In [162]:
base_url.replace("/sets","")+'/card/Shuckle-EX-Battle-Boost-EBB-1'

'https://jp.pokellector.com/card/Shuckle-EX-Battle-Boost-EBB-1'

In [194]:
bs = BeautifulSoup(card_r.text)

In [173]:
card_img

'https://den-cards.pokellector.com/129/Shuckle.EBB.1.png'

In [227]:
dict_card

{'JPN': 'ツボツボ', 'Rarity': 'Common', 'Set': 'EX Battle Boost', 'Card': '1/93'}

In [228]:
int('3o')

ValueError: invalid literal for int() with base 10: '3o'

In [246]:
card_name.contents[1]

'Pokemon Enterprise '

In [10]:
def process_one_card(card: BeautifulSoup, index: int, extension_code: str):
    try:
        card_page_request = requests.get(base_url.replace("/sets", "") + card.find('a')['href'])
        card_bs = BeautifulSoup(card_page_request.text, features="html.parser")
        card_info = card_bs.find('div', class_="infoblurb")
        card_img = card_bs.find('div', class_="card").find('img')['src']
        card_name = card_bs.find('h1', class_="icon set")
        card_str_name = card_name.contents[1].strip()

        dict_card = {}
        for card_meta in card_info.find_all('div'):
            val = (card_meta.text.split(": "))
            dict_card[val[0]] = val[1]

        try:
            card_japanese_name = dict_card['JPN']
        except KeyError:
            card_japanese_name = card_str_name

        try:
            card_rarity = dict_card['Rarity']
        except KeyError:
            card_rarity = "UNKNOWN"

        try:
            card_number = int(dict_card['Card'].split('/')[0])
        except (KeyError, ValueError):
            card_number = index + 1
            print(f"Problem with card, name became {card_name} and number is {card_number}")

        return [card_str_name, card_japanese_name, card_img, card_number, extension_code, card_rarity]
    except Exception as e:
        return None, e

In [86]:
nani = db.query_dataframe("SELECT * from pokemon.cardList WHERE cardName LIKE 'Gardenia\\'s Vigor'")

In [87]:
nani

,cardName,cardJapaneseName,cardExtensionCode,cardNumber,cardImageUrl,cardRarity
0,Gardenia's Vigor,ナタネの活気,S10D,65,https://den-cards.pokellector.com/341/Gardenia...,Uncommon
1,Gardenia's Vigor,Gardenia's Vigor,S10D,78,https://den-cards.pokellector.com/341/Gardenia...,Secret Rare
2,Gardenia's Vigor,Gardenia's Vigor,S10D,84,https://den-cards.pokellector.com/341/Gardenia...,Secret Rare


In [88]:
nani['cardName'] = nani['cardName'].apply(lambda x : x.replace("'", "\\'"))

In [89]:
nani

,cardName,cardJapaneseName,cardExtensionCode,cardNumber,cardImageUrl,cardRarity
0,Gardenia\'s Vigor,ナタネの活気,S10D,65,https://den-cards.pokellector.com/341/Gardenia...,Uncommon
1,Gardenia\'s Vigor,Gardenia's Vigor,S10D,78,https://den-cards.pokellector.com/341/Gardenia...,Secret Rare
2,Gardenia\'s Vigor,Gardenia's Vigor,S10D,84,https://den-cards.pokellector.com/341/Gardenia...,Secret Rare


In [191]:
df = db.query_dataframe("SELECT owned,cardExtensionCode FROM pokemon.cardList as cl LEFT JOIN pokemon.collection as co ON cl.cardName=co.cardName AND cl.cardNumber=co.cardNumber  ORDER BY cardNumber ASCENDING")

In [184]:
df[df['owned']==1]

,owned,cardExtensionCode
4,1,SI
36,1,S8AP
57,1,SI
144,1,SI
158,1,BW5
...,...,...
9923,1,S10D
10024,1,S10D
10067,1,S10D
10139,1,S10D


In [239]:
df_ext = db.query_dataframe("SELECT * from pokemon.extensions")

In [243]:
df_ext

,extensionCode,serieName,extensionName,extensionUrl,extensionImageUrl,extensionReleaseDate,extensionCardNumber,ownedNumber,percentage
0,BWP,Black & White Promos Series,Black & White Promos Set,https://jp.pokellector.com/sets/BWP-Black-Whit...,https://den-media.pokellector.com/logos/Black-...,2010-12-17 01:05:00+08:00,197,0.0,0.000000
1,KK,Black & White Promos Series,Blastoise & Kyurem Combo Deck Set,https://jp.pokellector.com/sets/KK-Blastoise-K...,https://den-media.pokellector.com/logos/Blasto...,2013-03-15 01:05:00+08:00,18,100.0,5.555556
2,DS,Black & White Promos Series,Dragon Selection Set,https://jp.pokellector.com/sets/DS-Dragon-Sele...,https://den-media.pokellector.com/logos/Dragon...,2012-01-27 01:05:00+08:00,20,0.0,0.000000
3,WAK,Black & White Promos Series,Exciting Battle for Everyone Set,https://jp.pokellector.com/sets/WAK-Exciting-B...,https://den-media.pokellector.com/logos/Exciti...,2012-11-16 01:05:00+08:00,47,0.0,0.000000
4,GBR,Black & White Promos Series,Garchomp Half Deck Set,https://jp.pokellector.com/sets/GBR-Garchomp-H...,https://den-media.pokellector.com/logos/Garcho...,2012-03-16 01:05:00+08:00,18,0.0,0.000000
...,...,...,...,...,...,...,...,...,...
153,S10D,Sword & Shield Series,Time Gazer Set,https://jp.pokellector.com/sets/S10D-Time-Gazer,https://den-media.pokellector.com/logos/Time-G...,2022-04-08 01:05:00+08:00,88,8800.0,100.000000
154,S7D,Sword & Shield Series,Towering Perfection Set,https://jp.pokellector.com/sets/S7D-Towering-P...,https://den-media.pokellector.com/logos/Perfec...,2021-07-09 01:05:00+08:00,90,0.0,0.000000
155,S8B,Sword & Shield Series,VMAX Climax Set,https://jp.pokellector.com/sets/S8B-VMAX-Climax,https://den-media.pokellector.com/logos/VMAX-C...,2021-12-03 01:05:00+08:00,28,0.0,0.000000
156,S1A,Sword & Shield Series,VMAX Rising Set,https://jp.pokellector.com/sets/S1A-VMAX-Rising,https://den-media.pokellector.com/logos/VMAX-R...,2020-02-07 01:05:00+08:00,86,0.0,0.000000


In [219]:
adf_ext[df_ext['ownedNJum']]

KeyError: "None of [Float64Index([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,\n              ...\n              nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],\n             dtype='float64', length=158)] are in the [columns]"

In [211]:
df_ext['percentage'] = df_ext[['extensionCode', 'extensionCardNumber']].apply(lambda x : group.loc[x[0]][0]/x[1] if x[0] in group.index else 0, axis=1)

In [212]:
df_ext[df_ext['percentage']!=0]

,serieName,extensionName,extensionCode,extensionUrl,extensionImageUrl,extensionReleaseDate,extensionCardNumber,percentage
1,Black & White Promos Series,Blastoise & Kyurem Combo Deck Set,KK,https://jp.pokellector.com/sets/KK-Blastoise-K...,https://den-media.pokellector.com/logos/Blasto...,2013-03-15 01:05:00+08:00,18,5.555556
12,Black & White Series,Cold Flare Set,BW6,https://jp.pokellector.com/sets/BW6-Cold-Flare,https://den-media.pokellector.com/logos/Cold-F...,2012-07-13 01:05:00+08:00,63,3.174603
14,Black & White Series,Dragon Blade Set,BW5,https://jp.pokellector.com/sets/BW5-Dragon-Blade,https://den-media.pokellector.com/logos/Dragon...,2012-03-16 01:05:00+08:00,53,3.773585
15,Black & White Series,Dragon Blast Set,BW5,https://jp.pokellector.com/sets/BW5-Dragon-Blast,https://den-media.pokellector.com/logos/Dragon...,2012-03-16 01:05:00+08:00,53,3.773585
17,Black & White Series,Freeze Bolt Set,BW6,https://jp.pokellector.com/sets/BW6-Freeze-Bolt,https://den-media.pokellector.com/logos/Freeze...,2012-07-13 01:05:00+08:00,63,3.174603
18,Black & White Series,Hail Blizzard Set,BW3,https://jp.pokellector.com/sets/BW3-Hail-Blizzard,https://den-media.pokellector.com/logos/Hail-B...,2011-09-16 01:05:00+08:00,55,3.636364
19,Black & White Series,Megalo Cannon Set,BW9,https://jp.pokellector.com/sets/BW9-Megalo-Cannon,https://den-media.pokellector.com/logos/Megalo...,2013-03-15 01:05:00+08:00,82,2.439024
21,Black & White Series,Psycho Drive Set,BW3,https://jp.pokellector.com/sets/BW3-Psycho-Drive,https://den-media.pokellector.com/logos/Psycho...,2011-09-16 01:05:00+08:00,55,3.636364
32,DPt Series,Shaymin LV.X Collection Pack Set,PTS,https://jp.pokellector.com/sets/PTS-Shaymin-LV...,https://den-media.pokellector.com/logos/Shaymi...,2009-04-18 01:05:00+08:00,12,8.333333
38,Neo Series,Awakening Legends Set,NEO3,https://jp.pokellector.com/sets/NEO3-Awakening...,https://den-media.pokellector.com/logos/Awaken...,2000-11-23 01:05:00+08:00,57,1.754386


In [175]:
test.fillna(0, inplace=True)

In [178]:
df[df['owned']==1]

,owned,cardExtensionCode
4,1,SI
36,1,S8AP
57,1,SI
144,1,SI
158,1,BW5
...,...,...
9923,1,S10D
10024,1,S10D
10067,1,S10D
10139,1,S10D


In [127]:
df[df['cardName']=='Gardenia\'s Vigor']

,cardExtensionCode,cardName,cardNumber,owned
8438,S10D,Gardenia's Vigor,65,0
9520,S10D,Gardenia's Vigor,78,0
9923,S10D,Gardenia's Vigor,84,0


In [118]:
db.execute("ALTER TABLE pokemon.collection UPDATE owned=1 where cardName='Gardenia'''s Vigor'")

ServerException: Code: 62.
DB::Exception: Syntax error: failed at position 74 ('s'): s Vigor'. Expected one of: DoubleColon, LIKE, GLOBAL NOT IN, end of query, DIV, IS, INTO OUTFILE, OR, QuestionMark, BETWEEN, NOT LIKE, MOD, AND, Comma, SETTINGS, IN, ILIKE, FORMAT, NOT ILIKE, NOT, token, NOT IN, GLOBAL IN. Stack trace:

0. DB::Exception::Exception(std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > const&, int, bool) @ 0x8fe8d5a in /usr/bin/clickhouse
1. DB::parseQueryAndMovePosition(DB::IParser&, char const*&, char const*, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > const&, bool, unsigned long, unsigned long) @ 0x115ed188 in /usr/bin/clickhouse
2. ? @ 0x10817a0a in /usr/bin/clickhouse
3. DB::executeQuery(std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > const&, std::__1::shared_ptr<DB::Context>, bool, DB::QueryProcessingStage::Enum, bool) @ 0x108175e3 in /usr/bin/clickhouse


In [140]:
db.query_dataframe("SELECT * FROM pokemon.cardList where cardName LIKE 'Gardenia''s%'")

,cardName,cardJapaneseName,cardExtensionCode,cardNumber,cardImageUrl,cardRarity
0,Gardenia's Vigor,ナタネの活気,S10D,65,https://den-cards.pokellector.com/341/Gardenia...,Uncommon
1,Gardenia's Vigor,Gardenia's Vigor,S10D,78,https://den-cards.pokellector.com/341/Gardenia...,Secret Rare
2,Gardenia's Vigor,Gardenia's Vigor,S10D,84,https://den-cards.pokellector.com/341/Gardenia...,Secret Rare


In [142]:
"fwej's".replace("'","''")

"fwej''s"

In [22]:

import requests
driver = new_driver = webdriver.Chrome(options=chrome_options,executable_path="C:\\Users\\maxpr\\Downloads\\chromedriver_win32\\chromedriver.exe" )
driver.get("https://jp.pokellector.com/sets/EBB-EX-Battle-Boost")
bs = BeautifulSoup(driver.page_source, features="html.parser")
driver.quit()
all_cards = bs.find_all('div' , class_="card")
cards = []
# Define a default card number in case a series has no card number
card_number = 0
r = Parallel(n_jobs=-1, backend="threading")(delayed(process_one_card)(card, index, 'LOL') for index, card in enumerate(all_cards))
# return pd.DataFrame(cards, columns=[CardList.CARD_NAME, CardList.CARD_IMAGE_URL, CardList.CARD_NUMBER,
#                                     CardList.CARD_EXTENSION_CODE, CardList.CARD_RARITY])

C:\Users\maxpr\AppData\Local\Temp\ipykernel_14428\2122781326.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = new_driver = webdriver.Chrome(options=chrome_options,executable_path="C:\\Users\\maxpr\\Downloads\\chromedriver_win32\\chromedriver.exe" )


In [19]:
r

[['Shuckle',
  'ツボツボ',
  'https://den-cards.pokellector.com/129/Shuckle.EBB.1.png',
  1,
  'LOL',
  'Common'],
 ['Snivey',
  'Snivey',
  'https://den-cards.pokellector.com/129/Snivey.EBB.2.png',
  2,
  'LOL',
  'Common'],
 ['Servine',
  'ジャノビー',
  'https://den-cards.pokellector.com/129/Servine.EBB.3.png',
  3,
  'LOL',
  'Common'],
 ['Serperior',
  'ジャローダ',
  'https://den-cards.pokellector.com/129/Serperior.EXB.4.png',
  4,
  'LOL',
  'Common'],
 ['Sewaddle',
  'クルミル',
  'https://den-cards.pokellector.com/129/Sewaddle.EBB.5.png',
  5,
  'LOL',
  'Common'],
 ['Swadloon',
  'クルマユ',
  'https://den-cards.pokellector.com/129/Swadloon.EBB.6.png',
  6,
  'LOL',
  'Common'],
 ['Leavanny',
  'ハハコモリ',
  'https://den-cards.pokellector.com/129/Leavanny.EBB.7.png',
  7,
  'LOL',
  'Common'],
 ['Virizion',
  'ビリジオン',
  'https://den-cards.pokellector.com/129/Virizion.EBB.8.png',
  8,
  'LOL',
  'Common'],
 ['Charmander',
  'ヒトカゲ',
  'https://den-cards.pokellector.com/129/Charmander.EBB.9.png',
  9,
 

In [256]:
pd.DataFrame(cards, columns=["name", "jp_name","url","number", "code", "number"])

,name,jp_name,url,number,code,number
0,Flareon VMAX,ブースター,https://den-cards.pokellector.com/330/Flareon-...,1,LOL,Common
1,Vaporeon VMAX,シャワーズ,https://den-cards.pokellector.com/330/Vaporeon...,2,LOL,Common
2,Jolteon VMAX,サンダース,https://den-cards.pokellector.com/330/Jolteon-...,3,LOL,Common
3,Espeon VMAX,エーフィ,https://den-cards.pokellector.com/330/Espeon-V...,4,LOL,Common
4,Fire Energy,Fire Energy,https://den-cards.pokellector.com/330/Fire-Ene...,5,LOL,Common
5,Electric Energy,Electric Energy,https://den-cards.pokellector.com/330/Electric...,6,LOL,Common
6,Psychic Energy,Psychic Energy,https://den-cards.pokellector.com/330/Psychic-...,7,LOL,Common
7,Water Energy,Water Energy,https://den-cards.pokellector.com/330/Water-En...,8,LOL,Common


In [133]:
%%time
data_blocks_2 = []
for idx, block in enumerate(blocks_bs):
    print(series[idx].text)
    r = Parallel(n_jobs=10, backend="threading")(delayed(test_bs)(extension) for extension in block.find_all('a'))
    #print(ext.text)
    #ext_name = 
    for res in r:
        data_blocks_2.append(res)
    # for extension in block.find_elements_by_class_name('button'):
    #     
df_ext_2 = pd.DataFrame(data_blocks_2, columns =["serie", "ext_code", "ext_url", "ext_name", "ext_img_url"])

Sword & Shield Series
S10A
/sets/S10A-Dark-Phantasma
Dark Phantasma Set
S10P
/sets/S10P-Space-Juggler
Space Juggler Set
S10D
/sets/S10D-Time-Gazer
Time Gazer Set
S9A
/sets/S9A-Battle-Region
Battle Region Set
S9
/sets/S9-Star-Birth
Star Birth Set
SI
/sets/SI-Start-Deck-100
Start Deck 100 Set
S8B
/sets/S8B-VMAX-Climax
VMAX Climax Set
S8A
/sets/S8A-25th-Anniversary-Collection
25th Anniversary Collection Set
S8AP
/sets/S8AP-25th-Anniversary-Promo-Pack
25th Anniversary Promo Pack Set
S8
/sets/S8-Fusion-ARTS
Fusion ARTS Set
nani
S7R
/sets/S7R-Blue-Sky-Stream
Blue Sky Stream Set
nani
S7D
/sets/S7D-Towering-Perfection
Towering Perfection Set
nani
S6A
/sets/S6A-Eevee-Heroes
Eevee Heroes Set
nani
S6K
/sets/S6K-Jet-Black-Spirit
Jet Black Spirit Set
nani
S6H
/sets/S6H-Silver-Lance
Silver Lance Set
nani
S5A
/sets/S5A-Matchless-Fighter
Matchless Fighter Set
nani
S5R
/sets/S5R-Rapid-Strike-Master
Rapid Strike Master Set
nani
S5I
/sets/S5I-Single-Strike-Master
Single Strike Master Set
nani
S4A
/sets/S

In [127]:
%%time
data_blocks = []
for idx, block in enumerate(blocks_bs):
    print(series[idx].text)
    for extension in block.find_all('a'):
        data_blocks.append(test_bs(extension))

df_ext = pd.DataFrame(data_blocks, columns =["serie", "ext_code", "ext_url", "ext_name", "ext_img_url"])

Sword & Shield Series
S10A
/sets/S10A-Dark-Phantasma
Dark Phantasma Set
nani
S10P
/sets/S10P-Space-Juggler
Space Juggler Set
nani
S10D
/sets/S10D-Time-Gazer
Time Gazer Set
nani
S9A
/sets/S9A-Battle-Region
Battle Region Set
nani
S9
/sets/S9-Star-Birth
Star Birth Set
nani
SI
/sets/SI-Start-Deck-100
Start Deck 100 Set
nani
S8B
/sets/S8B-VMAX-Climax
VMAX Climax Set
nani
S8A
/sets/S8A-25th-Anniversary-Collection
25th Anniversary Collection Set
nani
S8AP
/sets/S8AP-25th-Anniversary-Promo-Pack
25th Anniversary Promo Pack Set
nani
S8
/sets/S8-Fusion-ARTS
Fusion ARTS Set
nani
S7R
/sets/S7R-Blue-Sky-Stream
Blue Sky Stream Set
nani
S7D
/sets/S7D-Towering-Perfection
Towering Perfection Set
nani
S6A
/sets/S6A-Eevee-Heroes
Eevee Heroes Set
nani
S6K
/sets/S6K-Jet-Black-Spirit
Jet Black Spirit Set
nani
S6H
/sets/S6H-Silver-Lance
Silver Lance Set
nani
S5A
/sets/S5A-Matchless-Fighter
Matchless Fighter Set
nani
S5R
/sets/S5R-Rapid-Strike-Master
Rapid Strike Master Set
nani
S5I
/sets/S5I-Single-Strike-Mas

In [134]:
df_ext == df_ext_2

,serie,ext_code,ext_url,ext_name,ext_img_url
0,True,True,True,True,True
1,True,True,True,True,True
2,True,True,True,True,True
3,True,True,True,True,True
4,True,True,True,True,True
...,...,...,...,...,...
154,True,True,True,True,True
155,True,True,True,True,True
156,True,True,True,True,True
157,True,True,True,True,True


In [10]:
%%time
data_blocks = []
for idx, block in enumerate(blocks):
    print(series[idx].text)
    r = Parallel(n_jobs=4, backend="loky")(delayed(test)(extension) for extension in block.find_elements_by_class_name('button'))
    #print(ext.text)
    #ext_name = 
    for res in r:
        data_blocks.append(res)
    # for extension in block.find_elements_by_class_name('button'):
    #     
df_ext = pd.DataFrame(data_blocks, columns =["serie", "ext_code", "ext_url", "ext_name", "ext_img_url"])

Sword & Shield Series


PicklingError: Could not pickle the task to send it to the workers.

In [ ]:
%%time
data_blocks_2 = []
for idx, block in enumerate(blocks):
    print(series[idx].text)
    for extension in block.find_elements_by_class_name('button'):
        data_blocks_2.append(test(extension))
df_ext = pd.DataFrame(data_blocks_2, columns =["serie", "ext_code", "ext_url", "ext_name", "ext_img_url"])

Sword & Shield Series
S10A
https://jp.pokellector.com/sets/S10A-Dark-Phantasma
Dark Phantasma Set


c:\Users\maxpr\Documents\MyCode\pokemon-tcg-tracker\venv\lib\site-packages\selenium\webdriver\remote\webelement.py:464: UserWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  warnings.warn("find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead")
c:\Users\maxpr\Documents\MyCode\pokemon-tcg-tracker\venv\lib\site-packages\selenium\webdriver\remote\webelement.py:484: UserWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  warnings.warn("find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead")
C:\Users\maxpr\AppData\Local\Temp\ipykernel_10944\1135022650.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  new_driver = webdriver.Chrome(options=chrome_options,executable_path="C:\\Users\\maxpr\\Downl

S10P
https://jp.pokellector.com/sets/S10P-Space-Juggler
Space Juggler Set
S10D
https://jp.pokellector.com/sets/S10D-Time-Gazer
Time Gazer Set
S9A
https://jp.pokellector.com/sets/S9A-Battle-Region
Battle Region Set
S9
https://jp.pokellector.com/sets/S9-Star-Birth
Star Birth Set
SI
https://jp.pokellector.com/sets/SI-Start-Deck-100
Start Deck 100 Set
S8B
https://jp.pokellector.com/sets/S8B-VMAX-Climax
VMAX Climax Set
S8A
https://jp.pokellector.com/sets/S8A-25th-Anniversary-Collection
25th Anniversary Collection Set
S8AP
https://jp.pokellector.com/sets/S8AP-25th-Anniversary-Promo-Pack
25th Anniversary Promo Pack Set
S8
https://jp.pokellector.com/sets/S8-Fusion-ARTS
Fusion ARTS Set
S7R
https://jp.pokellector.com/sets/S7R-Blue-Sky-Stream
Blue Sky Stream Set
S7D
https://jp.pokellector.com/sets/S7D-Towering-Perfection
Towering Perfection Set
S6A
https://jp.pokellector.com/sets/S6A-Eevee-Heroes
Eevee Heroes Set
S6K
https://jp.pokellector.com/sets/S6K-Jet-Black-Spirit
Jet Black Spirit Set
S6H
ht

In [151]:
url_test = 'https://jp.pokellector.com/sets/S8-Fusion-ARTS'

In [152]:
driver.get(url_test)

In [153]:
card_metadata = driver.find_element_by_class_name("setinfo").text
print(card_metadata)

This set introduces new FUSION cards!
Cards
100
+29 Secret
Released
Sep 24th
2021


C:\Users\maxpr\AppData\Local\Temp/ipykernel_20160/3635570042.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  card_metadata = driver.find_element_by_class_name("setinfo").text


In [157]:
card_metadata = driver.find_element_by_class_name("setinfo").text
print(card_metadata)
card_metadata.split("Released")

This set introduces new FUSION cards!
Cards
100
+29 Secret
Released
Sep 24th
2021


C:\Users\maxpr\AppData\Local\Temp/ipykernel_20160/3946651833.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  card_metadata = driver.find_element_by_class_name("setinfo").text


['This set introduces new FUSION cards!\nCards\n100\n+29 Secret\n',
 '\nSep 24th\n2021']

In [155]:
all_cards = driver.find_element_by_class_name('cardlisting').find_elements_by_xpath('//div[@class="card "]')
cards = []
for card in all_cards:
    card_name = card.text
    card_img_ref = card.find_element_by_css_selector('a').find_element_by_css_selector('img').get_attribute('data-src')
    #print(card_name)
    #print(card_img_ref)
    cards.append([card_name, card_img_ref])
df_cards = pd.DataFrame(cards, columns = ['card_name','card_img_url'])

C:\Users\maxpr\AppData\Local\Temp/ipykernel_20160/4231425114.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  all_cards = driver.find_element_by_class_name('cardlisting').find_elements_by_xpath('//div[@class="card "]')
C:\Users\maxpr\AppData\Local\Programs\Python\Python38\lib\site-packages\selenium\webdriver\remote\webelement.py:426: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
C:\Users\maxpr\AppData\Local\Programs\Python\Python38\lib\site-packages\selenium\webdriver\remote\webelement.py:484: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


In [156]:
df_cards

,card_name,card_img_url
0,#1 - Caterpie,https://den-cards.pokellector.com/326/Caterpie...
1,#2 - Metapod,https://den-cards.pokellector.com/326/Metapod....
2,#3 - Butterfree,https://den-cards.pokellector.com/326/Butterfr...
3,#4 - Shroomish,https://den-cards.pokellector.com/326/Shroomis...
4,#5 - Breloom,https://den-cards.pokellector.com/326/Breloom....
...,...,...
124,#125 - Flaaffy,https://den-cards.pokellector.com/326/Flaaffy....
125,#126 - Power Tablet,https://den-cards.pokellector.com/326/Power-Ta...
126,#127 - Training Court,https://den-cards.pokellector.com/326/Training...
127,#128 - Grass Energy,https://den-cards.pokellector.com/326/Grass-En...


In [12]:
from tqdm.notebook import tqdm

def get_all_series(base_url: str) -> None:
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument("--window-size=1920,1080")
    driver = webdriver.Chrome(options=chrome_options,executable_path="C:\\Users\\maxpr\\Downloads\\chromedriver_win32\\chromedriver.exe" )
    driver.implicitly_wait(10)
    driver.get(base_url)
    series = driver.find_elements_by_class_name('set')
    blocks = driver.find_elements_by_class_name('buttonlisting')
    data_blocks = []
    for idx, block in enumerate(tqdm(blocks[:3])):
    
        #print(ext.text)
        #ext_name = 
        for extension in block.find_elements_by_class_name('button'):
            ext_code = extension.get_attribute('name')
            ext_url = extension.get_attribute('href')
            ext_name = extension.get_attribute('title')
            ext_image_url = extension.find_element_by_css_selector('img').get_attribute('src')
            #print(ext_code)
            #print(ext_url)
            #print(ext_name)
            df_card_ext = one_extension_cards(ext_url)
            df_card_ext['series'] = series[idx].text
            df_card_ext['ext_code'] = ext_code
            df_card_ext['ext_name'] = ext_name
            #data_blocks.append([series[idx].text, ext_code, ext_url, ext_name, ext_image_url])
            df_card_ext.to_csv(ext_name+".csv")
    #df_ext = pd.DataFrame(data_blocks, columns =["serie", "ext_code", "ext_url", "ext_name", "ext_img_url"])


def one_extension_cards(url_extension: str) -> pd.DataFrame:
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument("--window-size=1920,1080")
    driver = webdriver.Chrome(options=chrome_options,executable_path="C:\\Users\\maxpr\\Downloads\\chromedriver_win32\\chromedriver.exe" )
    driver.get(url_extension)
    card_metadata = driver.find_element_by_class_name("setinfo").text
    print(card_metadata)
    all_cards = driver.find_element_by_class_name('cardlisting').find_elements_by_xpath('//div[@class="card "]')
    cards = []
    for card in tqdm(all_cards):
        card_name = card.text
        card_img_ref = card.find_element_by_css_selector('a').find_element_by_css_selector('img').get_attribute('data-src')
        #print(card_name)
        #print(card_img_ref)
        cards.append([card_name, card_img_ref])
    df_cards = pd.DataFrame(cards, columns = ['card_name','card_img_url'])
    return df_cards

In [13]:
get_all_series(base_url)

C:\Users\maxpr\AppData\Local\Temp/ipykernel_23616/2609232765.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=chrome_options,executable_path="C:\\Users\\maxpr\\Downloads\\chromedriver_win32\\chromedriver.exe" )
C:\Users\maxpr\AppData\Local\Temp/ipykernel_23616/2609232765.py:13: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  series = driver.find_elements_by_class_name('set')
C:\Users\maxpr\AppData\Local\Temp/ipykernel_23616/2609232765.py:14: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  blocks = driver.find_elements_by_class_name('buttonlisting')


  0%|          | 0/3 [00:00<?, ?it/s]

C:\Users\maxpr\AppData\Local\Programs\Python\Python38\lib\site-packages\selenium\webdriver\remote\webelement.py:464: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
C:\Users\maxpr\AppData\Local\Programs\Python\Python38\lib\site-packages\selenium\webdriver\remote\webelement.py:484: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")
C:\Users\maxpr\AppData\Local\Temp/ipykernel_23616/2609232765.py:44: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=chrome_options,executable_path="C:\\Users\\maxpr\\Downloads\\chromedriver_win32\\chromedriver.exe" )


The Palkia half of the S10 set which introduces Hisuian Pokemon to the TCG
Cards
67
Released
Apr 8th
2022


C:\Users\maxpr\AppData\Local\Temp/ipykernel_23616/2609232765.py:46: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  card_metadata = driver.find_element_by_class_name("setinfo").text
C:\Users\maxpr\AppData\Local\Temp/ipykernel_23616/2609232765.py:48: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  all_cards = driver.find_element_by_class_name('cardlisting').find_elements_by_xpath('//div[@class="card "]')
C:\Users\maxpr\AppData\Local\Programs\Python\Python38\lib\site-packages\selenium\webdriver\remote\webelement.py:426: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")


...

This set, featuring Arceus, introduced VSTAR Pokemon to the TCG!
Cards
100
+27 Secret
Released
Jan 14th
2022


  0%|          | 0/127 [00:00<?, ?it/s]

This "set" full of reprints contains 414 unique cards and is distributed via 100 different 60-card decks!
Cards
414
+8 Secret
Released
Dec 17th
2021


  0%|          | 0/430 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Checking parsing of Marketplace

In [10]:
import requests

In [12]:
requests.get('https://api.cardmarket.com/ws/v1.0/authenticate/bfaD9xOU0SXBhtBP')

<Response [410]>

In [13]:
!pip install requests_oauthlib

You should consider upgrading via the 'C:\Users\maxpr\Documents\MyCode\pokemon-tcg-tracker\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [14]:
https://www.cardmarket.com/en/Pokemon/Products/Singles/Star-Birth?idRarity=0&sortBy=price_desc&perSite=20
https://www.cardmarket.com/en/Pokemon/Products/Singles/Battle-Region?idRarity=0&sortBy=price_desc&perSite=20
from requests_oauthlib import OAuth1Session

In [ ]:
https://www.cardmarket.com/en/Pokemon/Products/Singles/VMAX-Special-Set?idRarity=0&sortBy=price_desc&perSite=20

In [ ]:
oauth = OAuth1Session(
                    self.config["app_token"],
                    client_secret=self.config["app_secret"],
                    resource_owner_key=self.config["access_token"],
                    resource_owner_secret=self.config["access_token_secret"],
                    realm=url,
                )